In [1]:
import mxnet as mx
import logging
import numpy as np
import os, urllib2, time
import warnings
warnings.simplefilter('ignore',DeprecationWarning)
logging.basicConfig(level=logging.DEBUG)

In [2]:
import pickle as pkl

data = pkl.load(open('svhn_64_box_truncated.p', 'r'))

In [3]:
data

{'X_test': array([[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [ 80,  81,  84, ...,  71,  73,  72],
        ..., 
        [  0,   0,   0, ...,   0,   0,   0],
        [144, 144, 147, ..., 131, 131, 132],
        [  0,   0,   0, ...,  82,  82,  83]], dtype=uint8),
 'X_train': array([[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [ 61,  60,  64, ..., 152, 152, 152],
        ..., 
        [  0,   0,   0, ...,   0,   0,   0],
        [131, 128, 126, ..., 159, 158, 159],
        [200, 201, 204, ..., 188, 184, 186]], dtype=uint8),
 'y_test': array([[ 28.,  20.,   9.,  24.],
        [ 26.,  26.,  12.,  13.],
        [  5.,   5.,  55.,  54.],
        ..., 
        [ 24.,  25.,  16.,  15.],
        [ -7.,   5.,  78.,  54.],
        [ 14.,  12.,  37.,  40.]], dtype=float32),
 'y_train': array([[  18.,   16.,   29.,   32.],
        [  19.,   13.,   27.,   39.],
        [ -18.,   -6.,  100.,   77.],
        ...

In [4]:
import numpy as np
from numpy import newaxis

data['X_train'] = mx.nd.array(data['X_train'], dtype=np.float32)
data['X_test'] = mx.nd.array(data['X_test'], dtype=np.float32)
data_x_train = data['X_train'].reshape((-1,3,64,64))
data_x_valid = data['X_test'].reshape((-1,3,64,64))
data['y_train'] = mx.nd.array(data['y_train'])
data['y_test'] = mx.nd.array(data['y_test'])
data_y_train = data['y_train'] 
data_y_valid = data['y_test'] 
train_set = mx.io.NDArrayIter(data_x_train, data_y_train, batch_size=32)
valid_set = mx.io.NDArrayIter(data_x_valid, data_y_valid, batch_size=32)

In [5]:
print train_set.provide_label
print train_set.provide_data

[DataDesc[softmax_label,(32, 4L),<type 'numpy.float32'>,NCHW]]
[DataDesc[data,(32, 3L, 64L, 64L),<type 'numpy.float32'>,NCHW]]


In [11]:
data = mx.symbol.Variable(name="data")
label = mx.sym.Variable('softmax_label')
conv1_1 = mx.symbol.Convolution(data=data, kernel=(3, 3), pad=(1, 1), num_filter=32, name="conv1_1")
relu1_1 = mx.symbol.Activation(data=conv1_1, act_type="relu", name="relu1_1")
conv1_2 = mx.symbol.Convolution(data=relu1_1, kernel=(3, 3), pad=(1, 1), num_filter=32, name="conv1_2")
relu1_2 = mx.symbol.Activation(data=conv1_2, act_type="relu", name="relu1_2")
pool1 = mx.symbol.Pooling(data=relu1_2, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool1")
dropout1 = mx.symbol.Dropout(data=pool1, p=0.5,name="dropout1")

conv2_1 = mx.symbol.Convolution(data=dropout1, kernel=(3, 3), pad=(1, 1), num_filter=64, name="conv2_1")
relu2_1 = mx.symbol.Activation(data=conv2_1, act_type="relu", name="relu2_1")
conv2_2 = mx.symbol.Convolution(data=relu2_1, kernel=(3, 3), pad=(1, 1), num_filter=64, name="conv2_2")
relu2_2 = mx.symbol.Activation(data=conv2_2, act_type="relu", name="relu2_2")
pool2 = mx.symbol.Pooling(data=relu2_2, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool2")
dropout2 = mx.symbol.Dropout(data=pool2, p=0.5,name="dropout2")

conv3_1 = mx.symbol.Convolution(data=dropout2, kernel=(3, 3), pad=(1, 1), num_filter=128, name="conv3_1")
relu3_1 = mx.symbol.Activation(data=conv3_1, act_type="relu", name="relu3_1")
conv3_2 = mx.symbol.Convolution(data=relu3_1, kernel=(3, 3), pad=(1, 1), num_filter=128, name="conv3_2")
relu3_2 = mx.symbol.Activation(data=conv3_2, act_type="relu", name="relu3_2")
flatten = mx.symbol.Flatten(data = relu3_2, name = "flatten")
linear1 = mx.symbol.FullyConnected(data= flatten ,num_hidden=4, name='linear1')
pred = mx.sym.SoftmaxOutput(data=linear1, label=label, name='softmax')

'''
data = mx.symbol.Variable(name="data")
label = mx.sym.Variable('softmax_label')
conv1_1 = mx.symbol.Convolution(data=data, kernel=(3, 3), pad=(1, 1), num_filter=64, name="conv1_1")
relu1_1 = mx.symbol.Activation(data=conv1_1, act_type="relu", name="relu1_1")
conv1_2 = mx.symbol.Convolution(data=relu1_1, kernel=(3, 3), pad=(1, 1), num_filter=64, name="conv1_2")
relu1_2 = mx.symbol.Activation(data=conv1_2, act_type="relu", name="relu1_2")
pool1 = mx.symbol.Pooling(data=relu1_2, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool1")
dropout1 = mx.symbol.Dropout(data=pool1, p=0.5,name="dropout1")

conv2_1 = mx.symbol.Convolution(data=dropout1, kernel=(3, 3), pad=(1, 1), num_filter=96, name="conv2_1")
relu2_1 = mx.symbol.Activation(data=conv2_1, act_type="relu", name="relu2_1")
conv2_2 = mx.symbol.Convolution(data=relu2_1, kernel=(3, 3), pad=(1, 1), num_filter=96, name="conv2_2")
relu2_2 = mx.symbol.Activation(data=conv2_2, act_type="relu", name="relu2_2")
pool2 = mx.symbol.Pooling(data=relu2_2, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool2")

flatten = mx.symbol.Flatten(data = pool2, name = "flatten")
linear1 = mx.symbol.FullyConnected(data= flatten ,num_hidden=4, name='linear1')
pred = mx.sym.SoftmaxOutput(data=linear1, label=label, name='softmax')
'''

In [12]:
model = mx.mod.Module(pred)

In [13]:
ctx = mx.cpu(0)

In [14]:
model.fit(
        train_data          = train_set,
        eval_data           = valid_set,
        eval_metric         = mx.metric.MSE(),
        optimizer           = mx.optimizer.RMSProp(),
        initializer         = mx.init.Xavier(),
        num_epoch           = 10,
        batch_end_callback  = mx.callback.Speedometer(32,100))

INFO:root:Epoch[0] Batch [100]	Speed: 520.61 samples/sec	mse=1090.991327
INFO:root:Epoch[0] Train-mse=1051.128430
INFO:root:Epoch[0] Time cost=11.300
INFO:root:Epoch[0] Validation-mse=1057.351976
INFO:root:Epoch[1] Batch [100]	Speed: 522.83 samples/sec	mse=1095.490099
INFO:root:Epoch[1] Batch [200]	Speed: 524.87 samples/sec	mse=1116.667969
INFO:root:Epoch[1] Batch [300]	Speed: 522.37 samples/sec	mse=1095.521641
INFO:root:Epoch[1] Batch [400]	Speed: 526.50 samples/sec	mse=1079.460156
INFO:root:Epoch[1] Batch [500]	Speed: 515.30 samples/sec	mse=1114.426328
INFO:root:Epoch[1] Batch [600]	Speed: 518.94 samples/sec	mse=1054.968672
INFO:root:Epoch[1] Batch [700]	Speed: 527.48 samples/sec	mse=1103.516016
INFO:root:Epoch[1] Batch [800]	Speed: 526.91 samples/sec	mse=1041.345078
INFO:root:Epoch[1] Train-mse=1027.326613
INFO:root:Epoch[1] Time cost=50.844
INFO:root:Epoch[1] Validation-mse=1057.356858
INFO:root:Epoch[2] Batch [100]	Speed: 528.55 samples/sec	mse=1095.500309
INFO:root:Epoch[2] Batch